In [5]:
using DifferentialEquations, ODE, ODEInterfaceDiffEq

## Define the ThreeBody Problem
const threebody_μ = parse(Float64,"0.012277471"); const threebody_μ′ = 1 - threebody_μ

f = (t,u,du) -> begin
  # 1 = y₁
  # 2 = y₂
  # 3 = y₁'
  # 4 = y₂'
  D₁ = ((u[1]+threebody_μ)^2 + u[2]^2)^(3/2)
  D₂ = ((u[1]-threebody_μ′)^2 + u[2]^2)^(3/2)
  du[1] = u[3]
  du[2] = u[4]
  du[3] = u[1] + 2u[4] - threebody_μ′*(u[1]+threebody_μ)/D₁ - threebody_μ*(u[1]-threebody_μ′)/D₂
  du[4] = u[2] - 2u[3] - threebody_μ′*u[2]/D₁ - threebody_μ*u[2]/D₂
end


t₀ = 0.0; T = parse(Float64,"17.0652165601579625588917206249")
tspan = (t₀,2T)

prob = ODEProblem(f,[0.994, 0.0, 0.0, parse(Float64,"-2.00158510637908252240537862224")],tspan)

test_sol = TestSolution(T,[prob.u0])
abstols = 1./10.^(3:13)
reltols = 1./10.^(0:10);

using Plots; gr()

Plots.GRBackend()

See that it's periodic in the chosen timespan:

In [2]:
sol = solve(prob,abstol=1e-14,reltol=1e-14)
@show sol[1] - sol[end]
@show sol[end] - prob.u0;

sol[1] - sol[end] = [2.38168e-10,7.0885e-10,1.15649e-7,3.70698e-8]
sol[end] - prob.u0 = [-2.38168e-10,-7.0885e-10,-1.15649e-7,-3.70698e-8]


In [3]:
apr = appxtrue(sol,test_sol)
@show sol[end]
@show apr.u[end]
@show apr.errors

sol[end] = [0.994,-7.0885e-10,-1.15649e-7,-2.00159]
apr.u[end] = [0.994,-7.0885e-10,-1.15649e-7,-2.00159]
apr.errors = Dict(:final=>3.84165e-8)


Dict{Symbol,Float64} with 1 entry:
  :final => 3.84165e-8

This three-body problem is known to be a tough problem. Let's see how the algorithms fair at standard tolerances.

### 5th Order Runge-Kutta Methods


In [7]:
setups = [Dict(:alg=>DP5())
          #Dict(:alg=>ode45()) #fails
          Dict(:alg=>BS5())
          Dict(:alg=>Tsit5())
          Dict(:alg=>dopri5())];
wp = ode_workprecision_set(prob,abstols,reltols,setups;appxsol=test_sol,save_everystep=false,numruns=100)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 3.5 
 
 
 10 
 
 
 - 
 
 
 3.0 
 
 
 10 
 
 
 - 
 
 
 2.5 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 OrdinaryDiffEq.DP5 
 
 
 
 OrdinaryDiffEq.BS5 
 
 
 
 OrdinaryDiffEq.Tsit5 
 
 
 
 ODEInterfaceDiffEq.dopri5

#### Full save, but no dense

In [9]:
setups = [Dict(:alg=>DP5(),:dense=>false)
          #Dict(:alg=>ode45()) # Fails
          Dict(:alg=>BS5(),:dense=>false)
          Dict(:alg=>Tsit5(),:dense=>false)
          Dict(:alg=>dopri5())];
wp = ode_workprecision_set(prob,abstols,reltols,setups;appxsol=test_sol,numruns=100)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 3.5 
 
 
 10 
 
 
 - 
 
 
 3.0 
 
 
 10 
 
 
 - 
 
 
 2.5 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 OrdinaryDiffEq.DP5 
 
 
 
 OrdinaryDiffEq.BS5 
 
 
 
 OrdinaryDiffEq.Tsit5 
 
 
 
 ODEInterfaceDiffEq.dopri5

#### Dense

In [10]:
setups = [Dict(:alg=>DP5())
          #Dict(:alg=>ode45()) #fails
          Dict(:alg=>BS5())
          Dict(:alg=>Tsit5())
          Dict(:alg=>dopri5())];
wp = ode_workprecision_set(prob,abstols,reltols,setups;appxsol=test_sol,numruns=100)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 3.5 
 
 
 10 
 
 
 - 
 
 
 3.0 
 
 
 10 
 
 
 - 
 
 
 2.5 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 OrdinaryDiffEq.DP5 
 
 
 
 OrdinaryDiffEq.BS5 
 
 
 
 OrdinaryDiffEq.Tsit5 
 
 
 
 ODEInterfaceDiffEq.dopri5

In these tests we see that most of the algorithms are close,with `BS5` and `DP5` showing much better than `Tsit5`. `ode45` errors.

### Higher Order Algorithms

In [11]:
setups = [Dict(:alg=>DP5())
          Dict(:alg=>Vern6())
          Dict(:alg=>Vern7())
          Dict(:alg=>TanYam7())
          Dict(:alg=>Vern8())
          Dict(:alg=>DP8())
          Dict(:alg=>dop853())
          Dict(:alg=>Vern9())];
wp = ode_workprecision_set(prob,abstols,reltols,setups;appxsol=test_sol,save_everystep=false,numruns=100)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 OrdinaryDiffEq.DP5 
 
 
 
 OrdinaryDiffEq.Vern6 
 
 
 
 OrdinaryDiffEq.Vern7 
 
 
 
 OrdinaryDiffEq.TanYam7 
 
 
 
 OrdinaryDiffEq.Vern8 
 
 
 
 OrdinaryDiffEq.DP8 
 
 
 
 ODEInterfaceDiffEq.dop853 
 
 
 
 OrdinaryDiffEq.Vern9

In [15]:
setups = [Dict(:alg=>DP5())
          Dict(:alg=>Vern6())
          Dict(:alg=>Vern7())
          Dict(:alg=>TanYam7())
          Dict(:alg=>Vern8())
          Dict(:alg=>DP8())
          Dict(:alg=>dop853())
          Dict(:alg=>Vern9())];
wp = ode_workprecision_set(prob,abstols,reltols,setups;appxsol=test_sol,dense=false,numruns=100,verbose=false)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 OrdinaryDiffEq.DP5 
 
 
 
 OrdinaryDiffEq.Vern6 
 
 
 
 OrdinaryDiffEq.Vern7 
 
 
 
 OrdinaryDiffEq.TanYam7 
 
 
 
 OrdinaryDiffEq.Vern8 
 
 
 
 OrdinaryDiffEq.DP8 
 
 
 
 ODEInterfaceDiffEq.dop853 
 
 
 
 OrdinaryDiffEq.Vern9

In [11]:
setups = [Dict(:alg=>DP5())
          Dict(:alg=>Vern6())
          Dict(:alg=>Vern7())
          Dict(:alg=>TanYam7())
          Dict(:alg=>Vern8())
          Dict(:alg=>DP8())
          Dict(:alg=>dop853())
          Dict(:alg=>Vern9())];
wp = ode_workprecision_set(prob,abstols,reltols,setups;appxsol=test_sol,numruns=100)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 DiffEqBase.DP5 
 
 
 
 DiffEqBase.Vern6 
 
 
 
 DiffEqBase.Vern7 
 
 
 
 DiffEqBase.TanYam7 
 
 
 
 DiffEqBase.Vern8 
 
 
 
 DiffEqBase.DP8 
 
 
 
 DiffEqBase.dop853 
 
 
 
 DiffEqBase.Vern9

In this test we see `:Vern6` and `:Vern7` shine.

### Other Algorithms

Once again we separate ODE.jl because it fails:

In [16]:
setups = [Dict(:alg=>ode78())];
wp = ode_workprecision_set(prob,abstols,reltols,setups;appxsol=test_sol,dense=false,numruns=100)

DiffEqDevTools.WorkPrecisionSet(DiffEqDevTools.WorkPrecision[DiffEqDevTools.WorkPrecision(DiffEqBase.ODEProblem{Array{Float64,1},Float64,true,##3#4,Void,UniformScaling{Int64}}(#3,[0.994,0.0,0.0,-2.00159],(0.0,34.1304),nothing,UniformScaling{Int64}
1*I),[0.001,0.0001,1.0e-5,1.0e-6,1.0e-7,1.0e-8,1.0e-9,1.0e-10,1.0e-11,1.0e-12,1.0e-13],[1.0,0.1,0.01,0.001,0.0001,1.0e-5,1.0e-6,1.0e-7,1.0e-8,1.0e-9,1.0e-10],[1.07378,1.21788,0.838866,0.748896,0.748896,0.748896,0.748896,0.748896,0.748896,3.25787,1.63293],[0.00026845,0.000157697,0.000171204,9.72379e-5,9.31552e-5,0.000107744,9.38093e-5,9.86203e-5,0.000106674,0.000200646,0.000108832],"ODE.ode78",11)],1,[0.001,0.0001,1.0e-5,1.0e-6,1.0e-7,1.0e-8,1.0e-9,1.0e-10,1.0e-11,1.0e-12,1.0e-13],[1.0,0.1,0.01,0.001,0.0001,1.0e-5,1.0e-6,1.0e-7,1.0e-8,1.0e-9,1.0e-10],DiffEqBase.ODEProblem{Array{Float64,1},Float64,true,##3#4,Void,UniformScaling{Int64}}(#3,[0.994,0.0,0.0,-2.00159],(0.0,34.13043312031593),nothing,UniformScaling{Int64}
1*I),Dict{Symbol,ODE.ode78}[

In [17]:
setups = [Dict(:alg=>DP5())
          Dict(:alg=>Vern7())
          Dict(:alg=>TanYam7())
          Dict(:alg=>Vern8())
          Dict(:alg=>DP8())
          Dict(:alg=>odex())
          Dict(:alg=>CVODE_Adams()
    ];
wp = ode_workprecision_set(prob,abstols,reltols,setups;appxsol=test_sol,save_everystep=false,numruns=100)
plot(wp)

LoadError: syntax: unexpected "]" in argument list

Again, on cheap function calculations the Adams methods are shown to not be efficient once the error is sufficiently small. Also, as seen in other places, the extrapolation methods do not fare as well as the Runge-Kutta methods.

### Conclusion

As in the other tests, the OrdinaryDiffEq.jl algorithms with the Verner Efficient methods are the most efficient solvers at stringent tolerances for most of the tests, while the order 5 methods do well at cruder tolerances. However, these tests are all so close it's hard to call.